In [1]:
import pandas as pd
import matplotlib
import numpy as np
import seaborn 
import re
import random
import datetime

In [2]:
test_2klog_info = pd.read_csv("data\HDFS_2K\HDFS_2k.log_structured.csv")

In [3]:
log_lines = []
with open('data\\HDFS_2K\\HDFS_2k.log', 'r') as file:
    for line in file:
        log_lines.append(line.strip())

raw_logs = pd.DataFrame(log_lines, columns=['Log'])
raw_logs['LineId'] = range(1, 2001)



In [4]:
# temp = pd.read_table("data\HDFS_2K\HDFS_2k.log", header=None)
# temp

In [5]:
merged_logs = test_2klog_info.merge(raw_logs, on = "LineId")


In [18]:
merged_logs_small = merged_logs[["LineId", "EventTemplate", "Log"]]
merged_logs_small.head(1)

,LineId,EventTemplate,Log
0,1,PacketResponder <*> for block blk_<*> terminating,081109 203615 148 INFO dfs.DataNode$PacketResp...


In [7]:
#merged_logs_small.to_csv("data\\merged_logs_small.csv")

In [8]:
def Augment_Template(template):
  augmented_template = template.replace("<*>", "<*> type <*>" ).replace("caught_fire", "temp_exceeded")
  augmented_template += " and <*> of data_loss expected. No backup present"
  return augmented_template

In [23]:
def Generate_Random_Values(template):
    # Create a list to hold the generated values
    values = []
    
    # Find all placeholders
    placeholders = re.findall(r"<\*>", template)
    
    # Iterate over the found placeholders and generate corresponding values
    for i, placeholder in enumerate(placeholders):
        if i == 0:
            # First <*> corresponds to random selection between specific strings
            values.append(random.choice(["PacketResponder", "DataNode", "NameNode"]))
        elif i == 1:
            # Second <*> corresponds to current time in HH:MM format
            values.append(str(datetime.datetime.now().strftime("%H:%M")))
        elif i == 2:
            # Third <*> corresponds to a random integer between 0 and 100
            values.append(str(random.randint(0, 100)))
        elif i == 3:
            # Fourth <*> corresponds to a random integer between 0 and 100
            values.append(str(random.randint(0, 100)))
    
    # Replace the placeholders in the template with the generated values
    for value in values:
        template = template.replace("<*>", value, 1)
    
    return template

In [24]:
tes_log = merged_logs_small.iloc[0, 2]
test_template = merged_logs_small.iloc[0, 1]

a = Augment_Template(test_template)
a

'PacketResponder <*> type <*> for block blk_<*> type <*> terminating and <*> of data_loss expected. No backup present'

In [25]:
test_template

'PacketResponder <*> for block blk_<*> terminating'

In [26]:
for placeholder in re.findall(r"<\*>", a):
    if placeholder == "<*>":
        print(placeholder)

<*>
<*>
<*>
<*>
<*>


In [27]:
b = Generate_Random_Values(str(a))
b

'PacketResponder PacketResponder type 14:18 for block blk_5 type 80 terminating and <*> of data_loss expected. No backup present'